In [103]:
import pandas as pd
import numpy as np
import yfinance as yf

In [222]:
#Motley-fool data with transcript for different tickers (company code) and dates
earnings_call_data = pd.read_pickle(r"C:\Users\ruchi\OneDrive\Ruchira-MAIN\Work\Stats_ML\motley-fool-data.pkl\motley-fool-data.pkl")

#adjusting the date format and aligning with market closing time date
earnings_call_data['date'] = earnings_call_data['date'].str.replace(r'\s*ET$', '', regex=True)
earnings_call_data['date'] = earnings_call_data['date'].str.replace('a.m.', 'AM', regex=False).str.replace('p.m.', 'PM', regex=False).str.strip()
earnings_call_data['date'] = pd.to_datetime(earnings_call_data['date'])
earnings_call_data['adjusted_date'] = earnings_call_data['date'].apply(lambda x: x + pd.offsets.BDay(1) if x.hour >= 16 else x)
earnings_call_data['adjusted_date'] = earnings_call_data['adjusted_date'].dt.normalize()
earnings_call_data['adjusted_date']=pd.to_datetime(earnings_call_data['adjusted_date'])


C:\Users\ruchi\AppData\Local\Temp\ipykernel_33768\2414088161.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  earnings_call_data['date'] = pd.to_datetime(earnings_call_data['date'])


In [88]:
earnings_call_data.head()

,date,exchange,q,ticker,transcript,adjusted_date
0,2020-08-27 21:00:00,NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel...",2020-08-28
1,2020-07-30 16:30:00,NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...,2020-07-31
2,2019-10-23 17:00:00,NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...,2019-10-24
3,2019-11-06 12:00:00,NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon...",2019-11-06
4,2019-08-07 08:30:00,NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...,2019-08-07


In [80]:
#defining start and end date to get the time range for the dataset we have 
start_date=earnings_call_data['adjusted_date'].min().strftime('%Y-%m-%d')
end_date=earnings_call_data['adjusted_date'].max().strftime('%Y-%m-%d')
tickers = earnings_call_data['ticker'].unique().tolist()

In [142]:
start_date

'2019-04-11'

In [66]:
#getting the stock prices data from yfinance for the data range above
stock_prices = yf.download(tickers, start=start_date, end=end_date)


[                       0%                       ]  10 of 2876 completed$ATCO: possibly delisted; no timezone found
[                       0%                       ]  11 of 2876 completed$ONDK: possibly delisted; no timezone found
[                       0%                       ]  12 of 2876 completed$HMTV: possibly delisted; no timezone found
[                       1%                       ]  21 of 2876 completed$SWIM): possibly delisted; no timezone found
[                       1%                       ]  22 of 2876 completed$JT: possibly delisted; no timezone found
[                       1%                       ]  23 of 2876 completed$CSII: possibly delisted; no timezone found
[                       1%                       ]  33 of 2876 completed$ATTO: possibly delisted; no timezone found
[                       1%                       ]  34 of 2876 completed$CO: possibly delisted; no timezone found
[                       1%                       ]  35 of 2876 completed$VA

In [140]:
#extracting just the stock prices from the data set above
adjusted_close_prices = stock_prices['Close']
adjusted_close_prices.tail()

Ticker,A,AA,AAIC,AAL,AAN,AAOI,AAON,AAP,AAPL,AAT,...,ZLND.Y,ZM,ZNGA,ZS,ZTS,ZUMZ,ZUO,ZVIA,ZY,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-15,148.687714,46.684219,NaN,16.799999,NaN,2.83,51.950047,142.273804,NaN,22.068241,...,NaN,80.769997,NaN,142.770004,169.882965,25.530001,NaN,4.50,NaN,NaN
2023-02-16,145.287689,47.031399,NaN,16.389999,NaN,2.95,51.765823,141.239365,NaN,21.918573,...,NaN,77.910004,NaN,132.929993,168.824982,25.219999,NaN,4.49,NaN,NaN
2023-02-17,145.268158,45.218410,NaN,16.360001,NaN,2.80,52.634285,141.051300,NaN,21.868679,...,NaN,76.110001,NaN,132.350006,166.980743,25.309999,NaN,4.47,NaN,NaN
2023-02-21,140.525772,46.356346,NaN,15.690000,NaN,2.72,51.035526,135.596970,NaN,21.186844,...,NaN,72.470001,NaN,127.839996,164.855026,24.090000,NaN,4.24,NaN,NaN
2023-02-22,139.212814,47.233910,NaN,15.560000,NaN,2.82,51.338177,135.060944,NaN,21.403038,...,NaN,73.389999,NaN,132.160004,161.545090,24.320000,NaN,4.37,NaN,NaN


In [122]:
non_empty_close = []
for i in adjusted_close_prices.columns:
    if adjusted_close_prices[i].isna().sum() < 974:
        non_empty_close.append(i)

len(non_empty_close)

1678

In [124]:
len(adjusted_close_prices.columns)

2876

In [107]:
earnings_call_data['adj_close_price'] = earnings_call_data.apply(
    lambda row: adjusted_close_prices.loc[row['adjusted_date'], row['ticker']]
    if row['adjusted_date'] in adjusted_close_prices.index and row['ticker'] in adjusted_close_prices.columns
    else 0,
    axis=1
)

# Count NaNs
nan_count = earnings_call_data['adj_close_price'].isna().sum()
print(f"Number of NaNs: {nan_count}")
earnings_call_data.head()

Number of NaNs: 7304


,date,exchange,q,ticker,transcript,adjusted_date,adj_close_price
0,2020-08-27 21:00:00,NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel...",2020-08-28,0.0
1,2020-07-30 16:30:00,NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...,2020-07-31,0.0
2,2019-10-23 17:00:00,NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...,2019-10-24,0.0
3,2019-11-06 12:00:00,NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon...",2019-11-06,NaN
4,2019-08-07 08:30:00,NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...,2019-08-07,0.0


In [112]:
import datetime as dt
adjusted_close_prices.loc[dt.datetime(2019,11,6),"BBSI"]

np.float64(nan)

#### Only SP500

In [176]:
folder =r"C:\Users\ruchi\OneDrive\Ruchira-MAIN\Work\Stats_ML\stock_data"
start_date = earnings_call_data['adjusted_date'].min().strftime('%Y-%m-%d')
end_date = "2022-12-12"

dates = pd.date_range(start_date, end_date, freq="D")


In [164]:
dates    

DatetimeIndex(['2019-04-11', '2019-04-12', '2019-04-13', '2019-04-14',
               '2019-04-15', '2019-04-16', '2019-04-17', '2019-04-18',
               '2019-04-19', '2019-04-20',
               ...
               '2022-12-22', '2022-12-23', '2022-12-24', '2022-12-25',
               '2022-12-26', '2022-12-27', '2022-12-28', '2022-12-29',
               '2022-12-30', '2022-12-31'],
              dtype='datetime64[ns]', length=1361, freq='D')

In [154]:
import os
sp500_tickers = []
for file in os.listdir(folder):
    ticker = os.path.splitext(file)[0]
    sp500_tickers.append(ticker)
    required_tickers = [t for t in sp500_tickers if t in tickers]
    

In [167]:
series_by_ticker = {}

for t in required_tickers:
    path = os.path.join(folder, f"{t}.csv")
    df = pd.read_csv(path, parse_dates = ["Date"], infer_datetime_format=True)
    adj_price = df[["Date", "Adj Close"]]
    s = adj_price.set_index('Date')["Adj Close"].rename(t)
    series_by_ticker[t] = s


C:\Users\ruchi\AppData\Local\Temp\ipykernel_33768\2167643534.py:5: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(path, parse_dates = ["Date"], infer_datetime_format=True)
C:\Users\ruchi\AppData\Local\Temp\ipykernel_33768\2167643534.py:5: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(path, parse_dates = ["Date"], infer_datetime_format=True)
C:\Users\ruchi\AppData\Local\Temp\ipykernel_33768\2167643534.py:5: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict

In [168]:
prices_df = pd.DataFrame(series_by_ticker)


In [191]:
prices_df.index = pd.to_datetime(prices_df.index).normalize()
start = pd.to_datetime(start_date) - pd.tseries.offsets.BDay(5)
end = pd.to_datetime(end_date)

# slice by datetime range
new_df = prices_df.loc[start:end]
# Calculate 1-day and 5-day returns for each ticker
for ticker in new_df.columns:
    new_df[f'{ticker}_r1d'] = new_df[ticker].pct_change(1)
    new_df[f'{ticker}_r5d'] = new_df[ticker].pct_change(5)

C:\Users\ruchi\AppData\Local\Temp\ipykernel_33768\2087891987.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[f'{ticker}_r1d'] = new_df[ticker].pct_change(1)
C:\Users\ruchi\AppData\Local\Temp\ipykernel_33768\2087891987.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[f'{ticker}_r5d'] = new_df[ticker].pct_change(5)
C:\Users\ruchi\AppData\Local\Temp\ipykernel_33768\2087891987.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [194]:
new_df.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACGL,ACN,...,YUM_r1d,YUM_r5d,ZBH_r1d,ZBH_r5d,ZBRA_r1d,ZBRA_r5d,ZION_r1d,ZION_r5d,ZTS_r1d,ZTS_r5d
Date,,,,,,,,,,,,,,,,,,,,,
2019-04-04,78.834366,33.481113,165.644623,47.421318,69.048866,71.560455,282.869995,74.082596,32.470001,167.797302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-04-05,79.458572,33.609398,168.291122,47.738770,69.582504,72.785004,284.910004,74.440636,32.590000,168.734955,...,-0.004878,NaN,0.005136,NaN,0.020186,NaN,-0.002307,NaN,0.001373,NaN
2019-04-08,79.673141,33.431774,169.062195,48.489998,70.024452,73.585678,282.950012,73.988358,32.680000,169.454819,...,-0.003702,NaN,0.012185,NaN,0.014244,NaN,0.001051,NaN,0.001077,NaN
2019-04-09,79.409782,32.869320,167.215363,48.344593,68.948799,72.935715,281.649994,74.129677,32.110001,168.337189,...,-0.001104,NaN,-0.000777,NaN,-0.005961,NaN,-0.015120,NaN,-0.007532,NaN
2019-04-10,79.663391,33.569923,170.214096,48.616009,69.165596,69.704803,280.540009,74.412376,32.340000,169.034286,...,0.005629,NaN,0.001399,NaN,0.024941,NaN,0.005117,NaN,-0.001281,NaN


In [198]:
spy_series = yf.download("SPY", start=start.strftime("%Y-%m-%d"), end=(end + pd.Timedelta(days=1)).strftime("%Y-%m-%d"), progress=False)["Close"]

# normalize index, sort, compute returns
spy_series.index = pd.to_datetime(spy_series.index).normalize()
spy_series = spy_series.sort_index()


In [204]:
spy_series["m_r1d"] = spy_series["SPY"].pct_change(1)
spy_series["m_r5d"] = spy_series["SPY"].pct_change(5)
spy_series

Ticker,SPY,m_r1d,m_r5d
Date,,,
2019-04-04,259.408264,NaN,NaN
2019-04-05,260.663849,0.004840,NaN
2019-04-08,260.862579,0.000762,NaN
2019-04-09,259.525635,-0.005125,NaN
2019-04-10,260.410980,0.003411,NaN
...,...,...,...
2022-12-06,376.844666,-0.014415,-0.003542
2022-12-07,376.203583,-0.001701,-0.035616
2022-12-08,379.150726,0.007834,-0.027346


In [207]:
df_with_market = pd.merge(new_df, spy_series, left_index = True, right_index = True ,how = "left")

In [208]:
df_with_market

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACGL,ACN,...,ZBH_r5d,ZBRA_r1d,ZBRA_r5d,ZION_r1d,ZION_r5d,ZTS_r1d,ZTS_r5d,SPY,m_r1d,m_r5d
Date,,,,,,,,,,,,,,,,,,,,,
2019-04-04,78.834366,33.481113,165.644623,47.421318,69.048866,71.560455,282.869995,74.082596,32.470001,167.797302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259.408264,NaN,NaN
2019-04-05,79.458572,33.609398,168.291122,47.738770,69.582504,72.785004,284.910004,74.440636,32.590000,168.734955,...,NaN,0.020186,NaN,-0.002307,NaN,0.001373,NaN,260.663849,0.004840,NaN
2019-04-08,79.673141,33.431774,169.062195,48.489998,70.024452,73.585678,282.950012,73.988358,32.680000,169.454819,...,NaN,0.014244,NaN,0.001051,NaN,0.001077,NaN,260.862579,0.000762,NaN
2019-04-09,79.409782,32.869320,167.215363,48.344593,68.948799,72.935715,281.649994,74.129677,32.110001,168.337189,...,NaN,-0.005961,NaN,-0.015120,NaN,-0.007532,NaN,259.525635,-0.005125,NaN
2019-04-10,79.663391,33.569923,170.214096,48.616009,69.165596,69.704803,280.540009,74.412376,32.340000,169.034286,...,NaN,0.024941,NaN,0.005117,NaN,-0.001281,NaN,260.410980,0.003411,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-06,151.350006,14.330000,145.529999,142.910004,163.720001,169.320007,377.950012,103.860001,60.540001,285.410004,...,0.020084,-0.028992,-0.047260,-0.026435,-0.076230,-0.014678,0.035171,376.844666,-0.014415,-0.003542
2022-12-07,153.729996,13.550000,146.649994,140.940002,165.399994,170.289993,377.790009,104.809998,60.150002,286.690002,...,0.034055,-0.006569,-0.065636,-0.001909,-0.092049,-0.018295,-0.025237,376.203583,-0.001701,-0.035616
2022-12-08,156.279999,13.600000,146.589996,142.649994,165.990005,170.460007,380.750000,106.919998,60.560001,292.149994,...,0.020186,0.001109,-0.059519,0.007014,-0.088671,0.022829,-0.018897,379.150726,0.007834,-0.027346


In [215]:
for t in required_tickers:
    df_with_market[f'{t}_abn_r1d']=df_with_market[f'{t}_r1d']-df_with_market['m_r1d']
    df_with_market[f'{t}_abn_r5d']=df_with_market[f'{t}_r5d']-df_with_market['m_r5d']
    

In [223]:
# convert wide df_with_market columns like "TICK_abn_r1d" -> long table and merge
df_with_market.index = pd.to_datetime(df_with_market.index).normalize()
earnings_call_data['adjusted_date'] = pd.to_datetime(earnings_call_data['adjusted_date']).dt.normalize()

# 1d abnormal returns long
abn1 = df_with_market.filter(regex=r'_abn_r1d$').copy()
abn1.columns = [c.rsplit('_abn_r1d', 1)[0] for c in abn1.columns]   # rename cols -> ticker
abn1_long = abn1.stack().rename('abret_1d').reset_index()
abn1_long.columns = ['adjusted_date', 'ticker', 'abret_1d']

# 5d abnormal returns long
abn5 = df_with_market.filter(regex=r'_abn_r5d$').copy()
abn5.columns = [c.rsplit('_abn_r5d', 1)[0] for c in abn5.columns]
abn5_long = abn5.stack().rename('abret_5d').reset_index()
abn5_long.columns = ['adjusted_date', 'ticker', 'abret_5d']

# merge both into earnings_call_data
earnings_call_data = earnings_call_data.merge(
    abn1_long, on=['adjusted_date', 'ticker'], how='left'
).merge(
    abn5_long, on=['adjusted_date', 'ticker'], how='left'
)

In [224]:
earnings_call_data

,date,exchange,q,ticker,transcript,adjusted_date,abret_1d,abret_5d
0,2020-08-27 21:00:00,NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel...",2020-08-28,NaN,NaN
1,2020-07-30 16:30:00,NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...,2020-07-31,NaN,NaN
2,2019-10-23 17:00:00,NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...,2019-10-24,0.137401,0.121305
3,2019-11-06 12:00:00,NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon...",2019-11-06,NaN,NaN
4,2019-08-07 08:30:00,NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...,2019-08-07,NaN,NaN
...,...,...,...,...,...,...,...,...
18750,2021-11-09 13:00:00,NYSE: SWX,2021-Q3,SWX,Prepared Remarks:\nOperator\nLadies and gentle...,2021-11-09,NaN,NaN
18751,2021-11-18 12:00:00,NYSE: PNNT,2021-Q4,PNNT,"Prepared Remarks:\nOperator\nGood morning, and...",2021-11-18,NaN,NaN
18752,2022-02-08 11:00:00,NYSE: TDG,2022-Q1,TDG,Prepared Remarks:\nOperator\nThank you for sta...,2022-02-08,0.011638,0.038610
18753,2022-02-28 16:30:00,NASDAQ: DVAX,2021-Q4,DVAX,"Prepared Remarks:\nOperator\nGood day, ladies ...",2022-03-01,NaN,NaN


In [225]:
clean_df=earnings_call_data.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3546 entries, 2 to 18752
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           3546 non-null   datetime64[ns]
 1   exchange       3546 non-null   object        
 2   q              3546 non-null   object        
 3   ticker         3546 non-null   object        
 4   transcript     3546 non-null   object        
 5   adjusted_date  3546 non-null   datetime64[ns]
 6   abret_1d       3546 non-null   float64       
 7   abret_5d       3546 non-null   float64       
dtypes: datetime64[ns](2), float64(2), object(4)
memory usage: 249.3+ KB


In [239]:
clean_df['r1d_direction'] = (clean_df['abret_1d'] > 0).astype(int)
clean_df['r5d_direction'] = (clean_df['abret_5d'] > 0).astype(int)

C:\Users\ruchi\AppData\Local\Temp\ipykernel_33768\443728299.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['r1d_direction'] = (clean_df['abret_1d'] > 0).astype(int)
C:\Users\ruchi\AppData\Local\Temp\ipykernel_33768\443728299.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['r5d_direction'] = (clean_df['abret_5d'] > 0).astype(int)


In [240]:
final_df = clean_df[["adjusted_date" , "ticker", "transcript", "abret_1d" , "abret_5d","r1d_direction","r5d_direction"]]

In [241]:
final_df

,adjusted_date,ticker,transcript,abret_1d,abret_5d,r1d_direction,r5d_direction
2,2019-10-24,LRCX,Prepared Remarks:\nOperator\nGood day and welc...,0.137401,0.121305,1,1
11,2021-08-26,SJM,"Prepared Remarks:\nOperator\nGood morning, and...",-0.020495,-0.078457,0,0
19,2019-11-07,DXCM,Prepared Remarks:\nOperator\nWelcome to the De...,0.268035,0.246329,1,1
24,2021-07-01,WBA,Prepared Remarks:\nOperator\nLadies and gentle...,-0.079667,-0.077605,0,0
25,2021-05-04,ZBRA,"Prepared Remarks:\nOperator\nGood day, and wel...",-0.011428,-0.057492,0,0
...,...,...,...,...,...,...,...
18722,2021-09-14,ORCL,Prepared Remarks:\nOperator\nWelcome to Oracle...,-0.022728,-0.010115,0,0
18728,2022-03-17,DG,Prepared Remarks:\nOperator\nGood morning. My ...,0.031962,0.040319,1,1
18741,2021-08-06,VTR,"Prepared Remarks:\nOperator\nGood day, and tha...",-0.029662,-0.033833,0,0
18747,2022-08-25,NVDA,Prepared Remarks:\nOperator\nGood afternoon. M...,0.026006,-0.026226,1,0
